In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset = pd.read_csv('Data.tsv',sep='\t',quoting=3,names = ['keys','sub'])

#NOT USED
dataset['sub'] = dataset['sub'].astype('category')
dataset['sub'] = dataset['sub'].cat.codes
#for one hot encoding
dataset = pd.get_dummies(dataset, columns=["sub"])

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(1, 621286):
    line = dataset['keys'][i]
    keywords = line.split()
    keywords = [key.lower() for key in keywords]
    keywords = [ps.stem(word) for word in keywords if not word in set(stopwords.words('english'))]
    keywords = ' '.join(keywords)
    corpus.append(keywords)

In [ ]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus)
#X = X.toarray()

y = dataset.iloc[:, 1].values

In [ ]:
#Preparing to MultinomialNB
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [ ]:
#Preparing to Decistion Tree
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(random_state=0)

In [ ]:
#Preparing to Random Forests
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=0)

In [ ]:
#Fitting to Classifier
classifier.fit(X, y)

In [ ]:
#save model
import pickle
pickle.dump(classifier, open('classifier.pkl', 'wb'))
pickle.dump(cv, open('cv.pkl', 'wb'))

In [ ]:
#import model
import pickle
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=0)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
#loading model from file
classifier = pickle.load(open('classifier.pkl','rb'))
cv = pickle.load(open('cv.pkl','rb'))

In [ ]:
#For Prediction
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
query = 'android'
query = query.split()
query = [ps.stem(word) for word in query if not word in set(stopwords.words('english'))]
X_test = cv.transform([' '.join(query)])
ypred = classifier.predict(X_test.toarray())
ypred2 = classifier.predict_proba(X_test.toarray()).T
output = str(ypred[0])
classes = classifier.classes_.tolist()
subjects = [] 
while(True):
    val = ypred2.argmax(axis=0)
    print(val)
    if(ypred2[val][0] == 0):
        break
    subjects.append(val)
    ypred2[val]=[0]

#Subjects contains all predictions for subjects related to query
subjects = [classes[i[0]] for i in subjects]